In [1]:
import pandas as pd

In [12]:
df = pd.read_csv(r'D:\Big Data\python\pandas\Оконные.csv', sep=';')
df

,user_id,order_id,action,time
0,1,1,create_order,2022-08-24 01:52:21.000000
1,1,2,create_order,2022-08-24 01:58:22.000000
2,1,3,create_order,2022-09-24 01:58:23.000000
3,1,4,create_order,2022-09-02 05:52:21.000000
4,2,5,create_order,2022-09-03 05:52:22.000000
5,2,6,create_order,2022-09-04 05:52:21.000000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   6 non-null      int64 
 1   order_id  6 non-null      int64 
 2   action    6 non-null      object
 3   time      6 non-null      object
dtypes: int64(2), object(2)
memory usage: 320.0+ bytes


In [13]:
df.time = pd.to_datetime(df.time)
df.info()
# df.time.dt.day

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   user_id   6 non-null      int64         
 1   order_id  6 non-null      int64         
 2   action    6 non-null      object        
 3   time      6 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 320.0+ bytes


In [14]:
# Сортировка
df = df.sort_values(['user_id', 'time'])
df

,user_id,order_id,action,time
0,1,1,create_order,2022-08-24 01:52:21
1,1,2,create_order,2022-08-24 01:58:22
3,1,4,create_order,2022-09-02 05:52:21
2,1,3,create_order,2022-09-24 01:58:23
4,2,5,create_order,2022-09-03 05:52:22
5,2,6,create_order,2022-09-04 05:52:21


In [84]:
df.groupby('user_id').rank()
# df['number'] = df.groupby('user_id').rank()
# df

,order_id,time
0,1.0,1.0
1,2.0,2.0
3,4.0,3.0
2,3.0,4.0
4,1.0,1.0
5,2.0,2.0


### Когда был первый зааказ

In [15]:
# Данные схлопываются
df.groupby('user_id').min()['time']

user_id
1   2022-08-24 01:52:21
2   2022-09-03 05:52:22
Name: time, dtype: datetime64[ns]

Оконная функция .groupby + .transform

In [16]:
# select user_id, order_id, action, time,
# min(time) over(partition by user_id) as first_order_time
# from user_actions
# where action = 'create_order'
# limit 100
df['first_order_time'] = df.groupby('user_id')['time'].transform('min')
df

,user_id,order_id,action,time,first_order_time
0,1,1,create_order,2022-08-24 01:52:21,2022-08-24 01:52:21
1,1,2,create_order,2022-08-24 01:58:22,2022-08-24 01:52:21
3,1,4,create_order,2022-09-02 05:52:21,2022-08-24 01:52:21
2,1,3,create_order,2022-09-24 01:58:23,2022-08-24 01:52:21
4,2,5,create_order,2022-09-03 05:52:22,2022-09-03 05:52:22
5,2,6,create_order,2022-09-04 05:52:21,2022-09-03 05:52:22


### Сравнение текущего дня с предыдущим (по заказам)

In [17]:
df['date'] = df.time.dt.date
df

,user_id,order_id,action,time,first_order_time,date
0,1,1,create_order,2022-08-24 01:52:21,2022-08-24 01:52:21,2022-08-24
1,1,2,create_order,2022-08-24 01:58:22,2022-08-24 01:52:21,2022-08-24
3,1,4,create_order,2022-09-02 05:52:21,2022-08-24 01:52:21,2022-09-02
2,1,3,create_order,2022-09-24 01:58:23,2022-08-24 01:52:21,2022-09-24
4,2,5,create_order,2022-09-03 05:52:22,2022-09-03 05:52:22,2022-09-03
5,2,6,create_order,2022-09-04 05:52:21,2022-09-03 05:52:22,2022-09-04


In [18]:
df2 = df.groupby(['user_id', 'date'], as_index=False).agg({'order_id': 'count'})
# ИЛИ df2 = df.groupby(['user_id', 'date'], as_index=False)[['order_id']].count()
df2

,user_id,date,order_id
0,1,2022-08-24,2
1,1,2022-09-02,1
2,1,2022-09-24,1
3,2,2022-09-03,1
4,2,2022-09-04,1


In [37]:
# Общее кол-во заказов у пользователя
df2['total_orders'] = df2.groupby('user_id')['order_id'].transform('sum')
df2

,user_id,date,order_id,total_orders
0,1,2022-08-24,2,4
1,1,2022-09-02,1,4
2,1,2022-09-24,1,4
3,2,2022-09-03,1,2
4,2,2022-09-04,1,2


In [39]:
df2['last_day_orders'] = df2.groupby('user_id')['order_id'].shift(1)
df2
# select user_id, dt, orders,
# sum(orders) over(partition by user_id) as total_orders, ~ всего заказов у пользователя
# lag(orders, 1) over(partition by user_id order by dt) as last_day_orders
# from
# (select user_id, time::date as dt, count(distinct order_id) as orders
# from user_actions
# group by 1, 2 ~ user_id, date
# ) t1
# limit 100

,user_id,date,order_id,total_orders,last_day_orders
0,1,2022-08-24,2,4,NaN
1,1,2022-09-02,1,4,2.0
2,1,2022-09-24,1,4,1.0
3,2,2022-09-03,1,2,NaN
4,2,2022-09-04,1,2,1.0
